https://morvanzhou.github.io/tutorials/machine-learning/torch/4-05-DQN/

# 模块导入和参数设置

除了导入Torch的模块，我们还要导入Gym的模块

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym

#超参数
BATCH_SIZE=32
LR=0.01   #学习率
EPSILON=0.9   #最优选择动作百分比
GAMMA=0.9  #奖励递减参数
TARGET_REPLACE_ITER=100  # Q实现网络的更新频率
MEMORY_CAPACITY=2000     # 记忆库大小
env=gym.make('CartPole-v0')   # 立杆子游戏
env=env.unwrapped
N_ACTIONS=env.action_space.n   # 杆子能做的动作
N_STATES=env.observation_space.shape[0]  #杆子能获取的环境信息数

# 神经网络

DQN当中的神经网络模式，我们将依据这个模式建立两个神经网络，一个是现实网络（Target Net），一个是估计网络（Eval Net）。

In [2]:
class Net(nn.Module):
    def __init__(self,):
        super(Net,self).__init__()
        self.fc1=nn.Linear(N_STATES,10)
        self.fc1.weight.data.normal_(0,0.1)
        self.out=nn.Linear(10,N_ACTIONS)
        self.out.weight.data.normal_(0,0.1)
        
    def forward(self,x):
        x=self.fc1(x)
        x=F.relu(x)
        actions_value=self.out(x)
        return actions_value

# DQN体系

简化的DQN体系是这样，我们有两个net，有选动作机制，有存经历机制，有学习机制。

class DQN(object):
    def __init__(self):
        # 建立target net和eval net 还有 memory
        
    def choose_action(self,x):
        # 根据环境观测值选择动作的机制
        return action
    
    def store_transition(self,s,a,r,s_):
        #存储记忆
        
    def learn(self):
        # target 网络更新
        # 学习记忆库中的记忆

接下来就是具体的，在DQN中每个功能都是怎么做的：

In [3]:
class DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()

        self.learn_step_counter = 0     # 用于 target 更新计时
        self.memory_counter = 0         # 记忆库记数
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))     # 初始化记忆库
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)    # torch 的优化器
        self.loss_func = nn.MSELoss()   # 误差公式

    def choose_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        # 这里只输入一个 sample
        if np.random.uniform() < EPSILON:   # 选最优动作
            actions_value = self.eval_net.forward(x)
            action = torch.max(actions_value, 1)[1].data.numpy()[0, 0]     # return the argmax
        else:   # 选随机动作
            action = np.random.randint(0, N_ACTIONS)
        return action

    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        # 如果记忆库满了, 就覆盖老数据
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1

    def learn(self):
        # target net 参数更新
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1

        # 抽取记忆库中的批数据
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = torch.FloatTensor(b_memory[:, :N_STATES])
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int))
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2])
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:])

        # 针对做过的动作b_a, 来选 q_eval 的值, (q_eval 原本有所有动作的值)
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        q_next = self.target_net(b_s_).detach()     # q_next 不进行反向传递误差, 所以 detach
        q_target = b_r + GAMMA * q_next.max(1)[0]   # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)

        # 计算, 更新 eval net
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

# 训练

按照QLearning的形式进行off-policy的更新。我们进行回合制更新，一个回合完了，进入下一个回合。一直到他们将杆子立起来很久。

In [4]:
dqn=DQN()  #定义DQN系统

for i_episode in range(400):
    s=env.reset()
    while True:
        env.render() #显示实验动画
        a=dqn.choose_action(s)
        
        # 选动作，得到环境反馈
        s_,r,done,info=env.step(a)
        
        #修改reward，使DQN快速学习
        x,x_dot,theta,theta_dot=s_
        r1=(env.x_threshold-abs(x))/env.x_threshold-0.8
        r2=(env.theta_threshold_radians-abs(theta))/env.theta_threshold_radians-0.5
        r=r1+r2
        
        #存记忆
        dqn.store_transition(s,a,r,s_)
        
        if dqn.memory_counter>MEMORY_CAPACITY:
            dqn.learn()
            
        if done:
            break
            
        s=s_

IndexError: too many indices for array

# 完整程序

In [5]:
"""
View more, visit my tutorial page: https://morvanzhou.github.io/tutorials/
My Youtube Channel: https://www.youtube.com/user/MorvanZhou
More about Reinforcement learning: https://morvanzhou.github.io/tutorials/machine-learning/reinforcement-learning/
Dependencies:
torch: 0.4
gym: 0.8.1
numpy
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym

# Hyper Parameters
BATCH_SIZE = 32
LR = 0.01                   # learning rate
EPSILON = 0.9               # greedy policy
GAMMA = 0.9                 # reward discount
TARGET_REPLACE_ITER = 100   # target update frequency
MEMORY_CAPACITY = 2000
env = gym.make('CartPole-v0')
env = env.unwrapped
N_ACTIONS = env.action_space.n
N_STATES = env.observation_space.shape[0]
ENV_A_SHAPE = 0 if isinstance(env.action_space.sample(), int) else env.action_space.sample().shape     # to confirm the shape


class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_STATES, 50)
        self.fc1.weight.data.normal_(0, 0.1)   # initialization
        self.out = nn.Linear(50, N_ACTIONS)
        self.out.weight.data.normal_(0, 0.1)   # initialization

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value


class DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()

        self.learn_step_counter = 0                                     # for target updating
        self.memory_counter = 0                                         # for storing memory
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))     # initialize memory
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()

    def choose_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        # input only one sample
        if np.random.uniform() < EPSILON:   # greedy
            actions_value = self.eval_net.forward(x)
            action = torch.max(actions_value, 1)[1].data.numpy()
            action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index
        else:   # random
            action = np.random.randint(0, N_ACTIONS)
            action = action if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
        return action

    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        # replace the old memory with new memory
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1

    def learn(self):
        # target parameter update
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1

        # sample batch transitions
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = torch.FloatTensor(b_memory[:, :N_STATES])
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int))
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2])
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:])

        # q_eval w.r.t the action in experience
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        q_next = self.target_net(b_s_).detach()     # detach from graph, don't backpropagate
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)   # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

dqn = DQN()

print('\nCollecting experience...')
for i_episode in range(400):
    s = env.reset()
    ep_r = 0
    while True:
        env.render()
        a = dqn.choose_action(s)

        # take action
        s_, r, done, info = env.step(a)

        # modify the reward
        x, x_dot, theta, theta_dot = s_
        r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        r = r1 + r2

        dqn.store_transition(s, a, r, s_)

        ep_r += r
        if dqn.memory_counter > MEMORY_CAPACITY:
            dqn.learn()
            if done:
                print('Ep: ', i_episode,
                      '| Ep_r: ', round(ep_r, 2))

        if done:
            break
        s = s_


Ep:  176 | Ep_r:  2.53
Ep:  177 | Ep_r:  2.88
Ep:  178 | Ep_r:  2.9
Ep:  179 | Ep_r:  2.62
Ep:  180 | Ep_r:  1.56
Ep:  181 | Ep_r:  1.76
Ep:  182 | Ep_r:  1.3
Ep:  183 | Ep_r:  3.96
Ep:  184 | Ep_r:  4.05
Ep:  185 | Ep_r:  2.88
Ep:  186 | Ep_r:  5.1
Ep:  187 | Ep_r:  4.89
Ep:  188 | Ep_r:  0.9
Ep:  189 | Ep_r:  3.09
Ep:  190 | Ep_r:  1.27
Ep:  191 | Ep_r:  4.31
Ep:  192 | Ep_r:  3.54
Ep:  193 | Ep_r:  6.64
Ep:  194 | Ep_r:  2.19
Ep:  195 | Ep_r:  7.12
Ep:  196 | Ep_r:  2.66
Ep:  197 | Ep_r:  1.0
Ep:  198 | Ep_r:  3.23
Ep:  199 | Ep_r:  4.33
Ep:  200 | Ep_r:  5.24
Ep:  201 | Ep_r:  3.19
Ep:  202 | Ep_r:  10.83
Ep:  203 | Ep_r:  2.47
Ep:  204 | Ep_r:  23.4
Ep:  205 | Ep_r:  2.6
Ep:  206 | Ep_r:  1.35
Ep:  207 | Ep_r:  8.79
Ep:  208 | Ep_r:  11.04
Ep:  209 | Ep_r:  0.53
Ep:  210 | Ep_r:  1.7
Ep:  211 | Ep_r:  -6.74
Ep:  212 | Ep_r:  9.44
Ep:  213 | Ep_r:  8.67
Ep:  214 | Ep_r:  33.32
Ep:  215 | Ep_r:  244.52
Ep:  216 | Ep_r:  63.76
Ep:  217 | Ep_r:  94.96
Ep:  218 | Ep_r:  124.63
Ep:  21